# Data importation

In [1]:
import random
import numpy as np
from sklearn import svm
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
import pandas as pd
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import networkx as nx
import re

%matplotlib inline

In [2]:
info = pd.read_csv(
    "node_information.csv", 
    header= None, 
    names=["Id", "year", "title", "authors", "journal", "abstract"],
    sep=",",
    index_col = 0
)

In [3]:
X_train = pd.read_csv("training_set.txt", sep=" ", header=None)
X_test = pd.read_csv("testing_set.txt", sep=" ", header=None)
y_train = X_train[2]
X_train.drop([2], axis = 1, inplace = True)

print info.shape, X_train.shape, X_test.shape

(27770, 5) (615512, 2) (32648, 2)


In [4]:
######################
### FOR VALIDATION ###
######################


#####################
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2)
#####################

#####################
small_portion_to_train = 50000
small_portion_to_test  = 5000
#X_train = X_train[:small_portion_to_train]
#y_train = y_train[:small_portion_to_train]

#X_test  = X_test[:small_portion_to_test]
#y_test = y_test[:small_portion_to_test]
#####################

# Feature Preprocess

- list_authors is the list of authors in the papers
- list_universities is the list where the authors are from

In [5]:
def universities_to_keep(authors, universities):
    while('(' in authors and ')' in authors):
        universities.append( authors[authors.find('(')+1 : authors.find(')')] )
        authors = authors[: authors.find('(')] + authors[ authors.find(')')+1 : ]
            
    if '(' in authors:
        universities.append( authors[authors.find('(')+1 : ])
        authors = authors[: authors.find('(')]
    
    return authors, universities


def name_to_keep(author):
    if len(author.split(' ')) <= 1:
        return author
    
    while( author[0] == ' ' and len(author) > 0):
        author = author[1:]
    while( author[-1] == ' ' and len(author) > 0):
        author = author[:-1]
    
    author = author.replace('.', '. ')
    author = author.replace('.  ', '. ')
    name_to_keep = author.split(' ')[0][0] + '. ' + author.split(' ')[-1]

    return name_to_keep

# Transform concatenated names of authors to a list of authors 
list_authors = []
list_universities = []

info['authors'] = info['authors'].replace(np.nan, 'missing')
for authors in info['authors']:
    if authors != 'missing':
        ### split the different authors
        authors = authors.lower()
        
        ### Find the universities included in the name
        universities = []
        authors, universities = universities_to_keep(authors, universities)
        
        ### Split the authors
        authors = re.split(',|&', authors)
        
        ### For each author, check if university, and store it. Also, keep just the names (To be improved)
        authors_in_article = []      
        for author in authors:
            if author != ' ':
                authors_in_article.append(name_to_keep(author))
            
        list_universities.append(universities)
        list_authors.append(authors_in_article)
    else:
        list_universities.append(['missing'])
        list_authors.append(['missing'])   
        
info['authors'] = list_authors
info['universities'] = list_universities

# Topologic features

In [6]:
def make_graph(X_train, y_train, X_test):
    X_train = pd.concat([X_train, y_train], axis = 1)
    X_train = X_train.values
    G = nx.DiGraph()
    for i in range(X_train.shape[0]):
        source = X_train[i,0]
        target = X_train[i,1]
        G.add_node(source)
        G.add_node(target)
        if X_train[i,-1] == 1:
            G.add_edge(source,target)
            
    X_test = X_test.values
    for i in range(X_test.shape[0]):
        source = X_test[i,0]
        target = X_test[i,1]
        G.add_node(source)
        G.add_node(target)
        
    return G  

In [7]:
G = make_graph(X_train, y_train, X_test)  

In [8]:
def create_topologic_features(X, G):
    X_ = X.copy()
    X = X.values
    
    X_['Betweeness centrality'] = compute_betweeness_array(X, G)
    X_['Number common neighbours'] = make_common_neighbors(X, G)
    X_['Jaccard coefficienf'] = make_jaccard(X, G)
    diff_deg, to_deg = compute_diff_inlinks(X, G)
    X_['Difference in inlinks coefficient'] = diff_deg
    X_["Number of times to cited"] = to_deg
    X_['Same cluster'] = same_community(X,G)
    return X_

In [9]:
%%time
from create_topologic_features import create_topologic_features
X_train = create_topologic_features(X_train, G)
X_test = create_topologic_features(X_test, G)

Wall time: 1min


# Semantic features
- Cosine similarity within the titles as tf-idf
- Cosine similarity within the abstracts as tf-idf
- Cosine similarity within the titles as word2vec
- Cosine similarity within the abstracts as word2vec

### To try
- Difference cosine similarities?
- Keep the stopwords or not?
- Stemmise the words of not?

In [10]:
%%time
from semantic_features import *


sentences = []  # Initialize an empty list of sentences
word_list_abstract=pd.Series(index=info.index,dtype=np.str)
word_list_title=pd.Series(index=info.index,dtype=np.str)

for idx in info.index:
    w_list_a = text_to_sentences(info.loc[idx,"abstract"], tokenizer, True)
    sentences += w_list_a
    word_list_abstract[idx]= w_list_a
    w_list_t = text_to_sentences(info.loc[idx,"title"], tokenizer, True)
    sentences += w_list_t
    word_list_title[idx]= w_list_t

num_features = 200
model = load_model(sentences, train = False, num_features = num_features, min_word_count = 10, num_workers = 4, context = 10, downsampling = 1e-3)


centroid_title = dict(zip(info.index, getAvgFeatureVecs(info.title.values, model, num_features)))
centroid_abstract = dict(zip(info.index, getAvgFeatureVecs(info.abstract.values, model, num_features)))


# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

# Pre-allocate an array for the training set bags of centroids (for speed)
bag_centroids_abstract = {}
#centroids_title = {}
# Transform the training set reviews into bags of centroids
for review, idx in zip(word_list_abstract, word_list_abstract.index):
    bag_centroids_abstract[idx] = create_bag_of_centroids(review[0], word_centroid_map)
#for review, idx in zip(word_list_title, word_list_title.index):
#    centroids_title[idx] = create_bag_of_centroids( review[0], word_centroid_map )


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Igor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Wall time: 2min 44s


In [ ]:
w_l_title = info.title.apply(lambda x: text_to_wordlist(x, True))
w_l_abstract = info.abstract.apply(lambda x: text_to_wordlist(x, True))

In [69]:

from scipy.spatial.distance import cosine

   
def compute_cosines(centroids_dic, X):
    bag = []
    for idx, row in X.iterrows():
        
        condition = (np.isnan(centroids_dic[row[0]]).any() or np.isnan(centroids_dic[row[1]]).any())
        if condition:
            bag.append(0)
        else:
            bag.append(cosine(centroids_dic[row[0]], centroids_dic[row[1]]))

    return np.array(bag)

def create_nlp_features(X, centroid_title, centroid_abstract, bag_centroids_abstract, w_l_title, w_l_abstract):
    X_ = X.copy()
    
    X_['CosineD_title_centroid'] = compute_cosines(centroid_title,X)
    X_['CosineD_abstract_centroid'] = compute_cosines(centroid_abstract,X)
    X_['CosineD_abstract_bag_centroids'] = compute_cosines(bag_centroids_abstract,X)
    X_['Jaccard_title'] = compute_jaccard(w_l_title, X_)
    X_['Jaccard_abstract'] = compute_jaccard(w_l_abstract, X_)
    
    return X_


In [70]:
%%time
X_train = create_nlp_features(
    X_train, centroid_title, centroid_abstract, bag_centroids_abstract, w_l_title, w_l_abstract)
X_test = create_nlp_features(
    X_test,centroid_title, centroid_abstract, bag_centroids_abstract, w_l_title, w_l_abstract)

Wall time: 3min 49s


# Attribute features

- Difference in publication year
- Number of common authors
- Self-citation
- Same journal
- Number of times "to" cited (Attraction of the "to" paper)

### To try
- Number of times each author of "to" cited [Sum of these number of times] ?
- Number of times each journal cited?
- Number of same university??

In [72]:
%%time
from create_attribute_features import create_attribute_features
X_train = create_attribute_features(X_train,info)
X_test = create_attribute_features(X_test,info)

Wall time: 3min 44s


# Author Graph features

In [73]:
%%time 
from author_graph import make_graph_authors, create_topologic_features_authors

G_authors = make_graph_authors(X_train, y_train, info)
X_train = create_topologic_features_authors(X_train, G_authors, info, betweeness = True, common_neigh_and_jacc = True, inlinks = True)
X_test = create_topologic_features_authors(X_test, G_authors, info,  betweeness = True, common_neigh_and_jacc = True, inlinks = True)

Wall time: 6min 52s


# NEW FEATURES

In [ ]:
## Impact factors


# END

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_train=X_train.drop([0,1], axis = 1) 
X_test = X_test.drop([0,1], axis = 1)

In [ ]:
path = "validation_set2/"
X_train.to_csv(path+"X_train.csv")
X_test.to_csv(path+"X_test.csv")
y_train.to_csv(path+"y_train.csv")
y_test.to_csv(path+"y_test.csv")

# Classifier

In [75]:
def score(pred, real):
    tot = 0
    for i, val in enumerate(real):
        if pred[i] == val:
            tot += 1
    return float(tot)/len(real)

In [ ]:
from sklearn.base import BaseEstimator
from sknn.mlp import Layer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sknn.mlp import Classifier as MLP
from sklearn.pipeline import Pipeline

scaler1=StandardScaler()
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
nn = MLP(
layers=[
Layer("Rectifier", units=2000),
        Layer("Softmax")],
        learning_rule = 'momentum', learning_rate=0.02, batch_size = 150,dropout_rate = 0.25,
        n_iter=50,
        verbose = 1, 
        valid_size = 0.1, 
        n_stable = 15,
        debug = False,
        #    regularize = 'L2'
)


In [ ]:
clf = Pipeline([
    ("scaler", scaler1),
    ('neural network', nn)
    ])

clf.fit(X_train.drop([0,1], axis = 1), y_train)
pred = clf.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(clf.predict(X_train.drop([0,1], axis = 1)),y_train)

In [77]:
%%time 
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 500,n_jobs=4)
rfc.fit(X_train.drop([0,1], axis = 1), y_train)
pred = rfc.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(rfc.predict(X_train.drop([0,1], axis = 1)),y_train)

0.957718333428
1.0
Wall time: 5min 1s


In [ ]:
%%time 
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators = 400,n_jobs=2)
etc.fit(X_train.drop([0,1], axis = 1), y_train)
pred = etc.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(etc.predict(X_train.drop([0,1], axis = 1)),y_train)

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators = 200)
gbc.fit(X_train.drop([0,1], axis = 1), y_train)
pred = gbc.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(gbc.predict(X_train.drop([0,1], axis = 1)),y_train)

In [ ]:
%%time 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
clf =  Pipeline([
    ("scaler", MinMaxScaler()),
    ("regression",LogisticRegression(C=200000,max_iter = 1000))
    ])
clf.fit(X_train.drop([0,1], axis = 1), y_train)
pred = clf.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(clf.predict(X_train.drop([0,1], axis = 1)),y_train)

In [ ]:
def make_submission(predicted_label, name = 'submit.csv'):
    submit_d = d = {'id' : pd.Series(np.arange(1,X_test.shape[0]+1).astype(int)),
                    'category' : pd.Series(predicted_label).astype(int)}
    submit = pd.DataFrame(submit_d)
    submit.to_csv(name,index=False)
    return submit
